In [1]:
# Cell 0: Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Cell 1: 압축 해제
import zipfile
import os

zip_path = "/content/drive/MyDrive/Colab Notebooks/Movon/dataset/need_boundary/job_3906-2025_07_17_05_03_11-yolo 1.1.zip"
extract_dir = "/content/need_boundary_expanded"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ ZIP 압축 해제 완료:", extract_dir)

✅ ZIP 압축 해제 완료: /content/need_boundary_expanded


In [3]:
# 🔍 추출된 경로 내 라벨 및 이미지 존재 확인
from glob import glob

label_check = glob(os.path.join(extract_dir, "**", "*.txt"), recursive=True)
image_check = glob(os.path.join(extract_dir, "**", "*.jpg"), recursive=True)

if not label_check:
    raise FileNotFoundError("❌ 라벨(.txt) 파일을 찾을 수 없습니다.")
if not image_check:
    raise FileNotFoundError("❌ 이미지(.jpg) 파일을 찾을 수 없습니다.")

print(f"📄 라벨 파일 수: {len(label_check)}개")
print(f"🖼️ 이미지 파일 수: {len(image_check)}개")

📄 라벨 파일 수: 5001개
🖼️ 이미지 파일 수: 5000개


In [4]:
# Cell 2: YOLO 라벨 경계 확장 후처리 (margin = 0.008 기준)
from glob import glob

label_dir = os.path.join(extract_dir, "obj_train_data")  # CVAT YOLO 폴더
output_dir = os.path.join(label_dir, "labels_corrected")
os.makedirs(output_dir, exist_ok=True)

margin = 0.008  # 250px 기준 약 2픽셀 비율

def clamp(x):
    return max(0.0, min(1.0, x))

txt_files = glob(os.path.join(label_dir, "*.txt"))

for txt_file in txt_files:
    new_lines = []
    with open(txt_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            cls, x_c, y_c, w, h = map(float, parts)

            # YOLO → corner 좌표
            x1 = x_c - w / 2
            x2 = x_c + w / 2
            y1 = y_c - h / 2
            y2 = y_c + h / 2

            # 경계 확장
            if x1 < margin: x1 = 0.0
            if x2 > 1 - margin: x2 = 1.0
            if y1 < margin: y1 = 0.0
            if y2 > 1 - margin: y2 = 1.0

            # 다시 YOLO 포맷으로
            new_xc = (x1 + x2) / 2
            new_yc = (y1 + y2) / 2
            new_w = x2 - x1
            new_h = y2 - y1

            new_lines.append(f"{int(cls)} {clamp(new_xc):.6f} {clamp(new_yc):.6f} {clamp(new_w):.6f} {clamp(new_h):.6f}\n")

    # 저장
    out_path = os.path.join(output_dir, os.path.basename(txt_file))
    with open(out_path, 'w') as f:
        f.writelines(new_lines)

print(f"✅ 총 {len(txt_files)}개의 라벨 후처리 완료 (margin = {margin})")


✅ 총 5000개의 라벨 후처리 완료 (margin = 0.008)


In [5]:
# Cell 3: 이미지 + 후처리된 라벨 zip으로 압축
from shutil import copyfile, make_archive

export_dir = "/content/final_cvat_ready"
os.makedirs(export_dir, exist_ok=True)

# 이미지 및 라벨 복사
img_files = glob(os.path.join(label_dir, "*.jpg"))
for img_path in img_files:
    base = os.path.splitext(os.path.basename(img_path))[0]
    txt_path = os.path.join(output_dir, base + ".txt")

    copyfile(img_path, os.path.join(export_dir, base + ".jpg"))
    copyfile(txt_path, os.path.join(export_dir, base + ".txt"))

# 압축
final_zip_path = "/content/drive/MyDrive/Colab Notebooks/Movon/dataset/movon_dataset_corrected.zip"
make_archive(final_zip_path.replace(".zip", ""), 'zip', export_dir)

print("✅ 최종 ZIP 생성 완료:", final_zip_path)

✅ 최종 ZIP 생성 완료: /content/drive/MyDrive/Colab Notebooks/Movon/dataset/movon_dataset_corrected.zip


In [6]:
from google.colab import files
files.download(final_zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>